In [ ]:
import csv
import pandas as pd
import requests
from bs4 import BeautifulSoup
from bs4.element import NavigableString, Tag
import re
import datetime
import time
import jpholiday

df = pd.DataFrame()
cols = []

#最初のループでしか, 機能させない. カラム定義
def define_dataframe_colms(tables):
    global cols
    check_columns = [] 
    for tr in tables:
        if type(tr) is not  NavigableString:
            for factor in tr:
                if type(factor) is not NavigableString:
                    check_columns = factor.find_all('th', attrs={'class': re.compile(r'^t_.*$') }) #カラム情報
                    for check_column in check_columns: 
                        cols.append(check_column.get_text())
    if len(check_columns) > 0 :
        df = pd.DataFrame(columns=cols)

#アトラクション別-待ち時間の追加
def add_attraction_timedata(attraction_time_datas, date):
    global df #global変数排除すべき
    tmp_list = []
    tmp_df = pd.DataFrame()
    print(attraction_time_datas)
    for attraction_time_data in attraction_time_datas:
          if (attraction_time_data.get_text()) != "時間":
               tmp_list.append(attraction_time_data.get_text())
    if len(tmp_list) > 0 :
        tmp_df = pd.DataFrame(tmp_list).T
        tmp_df.columns = cols
        #tmp_dfに日毎の処理追加. 日付 + 時刻
        date = datetime.datetime.strptime(str(date), '%Y%m%d')
        date = date.strftime('%Y-%m-%d')
        tmp_df['時間'] = date + " " + tmp_df["時間"]
         
    df = pd.concat([df, tmp_df])
    
#テーブルをparseし, 必要情報の取得
def parse_tables(tables, date):
    for tr in tables:
        if type(tr) is not  NavigableString:  
            for factor in tr: 
                if type(factor) is not NavigableString:  
                    attraction_time_datas = factor.find_all(["th", "td"], attrs={'class': re.compile(r'^(?!t_).*$')}) 
                    add_attraction_timedata(attraction_time_datas, date)

def main():
    global df
    #日付は月ごと指定
    initial_value = 20190801
    cnt = initial_value
    for date in range (initial_value, 20240531):
        urlName = "https://urtrip.jp/tds-past-info/?rm=" + str(date) + "#page_top"
        time.sleep(0.1) #サーバ処理負荷軽減のため, 各アクセス0.1秒は空ける
        url = requests.get(urlName)
        soup = BeautifulSoup(url.content, "html.parser")
        try:
            tables = soup.findAll("table", "t_cool")[0]
            if date == initial_value: 
                define_dataframe_colms(tables)
            parse_tables(tables, date) 
        except:
            pass

    print("------------Finish!!-------------------") 
    # datetimeへ変換
    df['時間'] =  pd.to_datetime(df["時間"])  
    # 月曜日=0, 日曜日=6
    df['曜日_数値'] =  df["時間"].dt.weekday
    df['曜日'] =  df["時間"].dt.day_name() 
    #休日判定
    df['休日'] = df.時間.map(jpholiday.is_holiday)  
    #運休 / 案内終了は0分とする
    for df_col in cols:
        if df_col != "時間":
            df.loc[(df[df_col] == '案内終了') | (df[df_col] =='－') | (df[df_col] =='一時運休') | (df[df_col] == '計画運休'), [df_col]] = 0
            df[df_col]= df[df_col].astype('int') 
    print(df)
    print(df.dtypes)
    df.to_csv('DisneySea_waittime.csv')
if __name__ == "__main__":
    main()
